# ANÁLISE DOS REGISTROS DO SISTEMA COMANDO DE OCORRÊNCIAS COM ORIGEM METEOROLÓGICA

---

### Notebook Sections:
1. Exploratory Data Analysis
    1. Asses general quality of the dataset
    2. Check potential of the dataset to serve as catalog of incidents caused by rain, for predicive modeling.
2. Data Cleaning

### Importar modulos e funções

In [1]:
import os, pandas as pd, numpy as np, matplotlib.pyplot as plt, requests, json, folium
import seaborn as sns; sns.set()
from folium import plugins
from IPython.display import Image

### Definir classe 'data' com endereço dos dados
class data:
    path = r'C:\Users\luisr\Desktop\Repositories\Dados\Desafio COR-Rio IV\\'
    AlertaAPI = r'http://websempre.rio.rj.gov.br/json/chuvas'

### Carregando dados

In [40]:
comando = pd.read_csv(data.path + 'comando.csv')
data = comando.copy()

c:\Users\luisr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


---
# Data Cleaning

## 0. Extract records of incidents caused by rain

In [73]:
titles = [
    "Bolsão d'água em via", 'Vazamento de água / esgoto',
    'Alagamentos e enchentes', "Lâmina d'água",
    "Lâmina d'água em via", 'Alagamento',
    'Enchente', 'Bueiro'
]

records = comando[comando['POP_TITULO'].isin(titles)]
data = records.copy()
events = records.groupby('EVENTO_ID').first() # Isolando ocorrências (primeiro registro de cada ocorrência)

print(records.shape) # Número total de registros encontrados
print(events.shape) # Número eventos de registros encontrados

(12409, 18)
(4884, 17)


### 1. Asses data types

##### Check first rows

In [69]:
records.head(3)

,EVENTO_ID,EVENTO_TITULO,EVENTO_DESCRICAO,EVENTO_GRAVIDADE,EVENTO_BAIRRO,STATUS,EVENTO_INICIO,EVENTO_INICIO_HORA,EVENTO_FIM,EVENTO_FIM_HORA,EVENTO_PRAZO,EVENTO_LATITUDE,EVENTO_LONGITUDE,POP_TITULO,POP_DESCRICAO,ORGAO_SIGLA,ORGAO_NOME,ACAO
214,60,"Queda de Árvore na Rua Pacheco Leão, 1587",NaN,BAIXO,NaN,FECHADO,2015-04-10,15:59:00,2015-04-11,08:37:00,NaN,-23.0,-4.323430e+15,Bolsão d'água em via,Bolsão d'água em via,CET-RIO,Companhia de Engenharia de Tráfego,Desfazer o acidente
215,60,"Queda de Árvore na Rua Pacheco Leão, 1587",NaN,BAIXO,NaN,FECHADO,2015-04-10,15:59:00,2015-04-11,08:37:00,NaN,-23.0,-4.323430e+15,Bolsão d'água em via,Bolsão d'água em via,CET-RIO,Companhia de Engenharia de Tráfego,Organizar o trânsito
216,60,"Queda de Árvore na Rua Pacheco Leão, 1587",NaN,BAIXO,NaN,FECHADO,2015-04-10,15:59:00,2015-04-11,08:37:00,NaN,-23.0,-4.323430e+15,Bolsão d'água em via,Bolsão d'água em via,COMLURB,Companhia de Limpeza Urbana,Cortar e retirar árvore


##### Identify variables' types

In [70]:
cols = {
    'text': [
        'EVENTO_TITULO', 'EVENTO_DESCRICAO'
    ],
    'categorical': [
        'EVENTO_GRAVIDADE', 'EVENTO_BAIRRO', 'STATUS', 'EVENTO_PRAZO',
        'POP_TITULO', 'POP_DESCRICAO', 'ORGAO_SIGLA', 'ORGAO_NOME', 'ACAO'
    ],
    'datetime': [
        'EVENTO_INICIO', 'EVENTO_INICIO_HORA',
        'EVENTO_FIM', 'EVENTO_FIM_HORA'
    ],
    'location': [
        'EVENTO_LATITUDE', 'EVENTO_LONGITUDE'
    ]
}

#### Check raw data types

In [71]:
records.head().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 214 to 250
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EVENTO_ID           5 non-null      int64  
 1   EVENTO_TITULO       5 non-null      object 
 2   EVENTO_DESCRICAO    1 non-null      object 
 3   EVENTO_GRAVIDADE    5 non-null      object 
 4   EVENTO_BAIRRO       0 non-null      object 
 5   STATUS              5 non-null      object 
 6   EVENTO_INICIO       5 non-null      object 
 7   EVENTO_INICIO_HORA  5 non-null      object 
 8   EVENTO_FIM          5 non-null      object 
 9   EVENTO_FIM_HORA     5 non-null      object 
 10  EVENTO_PRAZO        0 non-null      object 
 11  EVENTO_LATITUDE     5 non-null      float64
 12  EVENTO_LONGITUDE    5 non-null      float64
 13  POP_TITULO          5 non-null      object 
 14  POP_DESCRICAO       5 non-null      object 
 15  ORGAO_SIGLA         5 non-null      object 
 16  ORGAO_NO

Conclusion: Only datetime variables require type conversion.

---
### 2. Asses categorical values

Categorical columns (unique categories):
* EVENTO_GRAVIDADE (BAIXO, MEDIO, ALTO, SEM_CLASSIFICAÇÃO, CRITICO). Extra values: Litoral do Rio de Janeiro
* EVENTO_BAIRRO (339) - Desestruturado
    * contém 'nan'
    * Valores de categorias não uniforme: Bairros de mesmo nome com escrita diferente.
    * Contem valores inválidos: Nomes de rua, números, endereços, etc.
    * Necessita método strip()
* EVENTO_STATUS (FECHADO, ABERTO). Extra values: FINALIZADO, Litoral do Rio de Janeiro
* (CURTO, MEDIO, LONGO)
* POP_TITULO (40)
* POP_DESCRICAO (42)
* ORGAO_SIGLA (34)
* ORGAO_NOME (35)
* ACAO (70)

In [ ]:
for col in cols['categorical']: # descomente para conferir valores unicos
    print('\n\n', col, ': ', len(records[col].unique()), '\n')
    display(records[col].value_counts())

---
### 3. Assess text variables

---
### 4. Asses location variables

#### Functions to compute and update order of magnitude of coordinates.

In [7]:
def orderOfMagnitude(number):
    return np.floor(np.log10(abs(number)))

def correct_magnitude(number, mag=1):
    if type(number)==float:
        magnitude = orderOfMagnitude(number)
        return number / 10 ** ( orderOfMagnitude(number) - mag )
    else:
        return [correct_magnitude(n, mag) for n in number]

nan_list_filter = lambda coord: not np.isnan(coord[0]) and not np.isnan(coord[1])
incomplete_values = [-22.8, -22.9, -23.0]
incomplete_coord_filter = lambda coord: coord[0] not in incomplete_values and coord[1] not in incomplete_values

In [74]:
data['lat'] = correct_magnitude(data['EVENTO_LONGITUDE'], mag=1)
data['lng'] = correct_magnitude(data['EVENTO_LATITUDE'], mag=1)


<ipython-input-7-cf9ef71831ff>:2: RuntimeWarning: divide by zero encountered in log10
  return np.floor(np.log10(abs(number)))
<ipython-input-7-cf9ef71831ff>:7: RuntimeWarning: invalid value encountered in double_scalars
  return number / 10 ** ( orderOfMagnitude(number) - mag )


In [127]:
def keep_from_decimal(numbers, decimal=1): # accepts array
    valid_index = []
    for limit in range(1, decimal+1):
        next_decimal = numbers * 10 ** (limit-1)
        next_abs_dif = abs(next_decimal - next_decimal.round(0))
        valid_index += list(numbers.index[next_abs_dif > 0.0])
    return np.unique(valid_index)

valid_index = keep_from_decimal(data['lat'], 3)
data['lat'].tolist()

### 5. Asses time variables

### 7. Check missing values notation uniformity

---
# Data Cleaning and Feature Engineering

---
## Location variables

##### Funções de tratamento/formatação de coordenadas

##### Tratando dados de coordenadas

In [ ]:
comando['lat'] = correct_magnitude(events['EVENTO_LONGITUDE'], mag=1)
comando['lng'] = correct_magnitude(events['EVENTO_LATITUDE'], mag=1)
                 
coords_raw = list(zip(
    comando['lat'].tolist(),
    events['lng'].tolist()
))

coords = list(filter(nan_list_filter, coords_raw))
coords = list(filter(incomplete_coord_filter, coords))

n_markers = 150

coords_txt = '|'.join( list(map(encode_coords, coords))[10: n_markers] )

len(coords_raw) - len(coords) # coordenadas faltando

---
# Exploratory data analysis: Catalog of incidents

In [4]:
titles = [
    "Bolsão d'água em via", 'Vazamento de água / esgoto',
    'Alagamentos e enchentes', "Lâmina d'água",
    "Lâmina d'água em via", 'Alagamento',
    'Enchente', 'Bueiro'
]

records = comando[comando['POP_TITULO'].isin(titles)]

events = records.groupby('EVENTO_ID').first() # Isolando ocorrências (primeiro registro de cada ocorrência)

print(records.shape) # Número total de registros encontrados
print(events.shape) # Número eventos de registros encontrados

(12409, 18)
(4884, 17)
